## Example backbone initialization

In [8]:
import unipercept as up
import torch

inp = torch.randn(16, 3, 512, 256).cuda()


def print_shapes(tensor_dict):
    for k, v in tensor_dict.items():
        print(k, v.shape)


bb = up.nn.backbones.timm.TimmBackbone(name="resnet50").cuda()
print_shapes(bb(inp))
%timeit out= bb(inp)
del bb

bb = up.nn.backbones.timm.TimmBackbone(name="resnet50", jit_script=True).cuda()
print_shapes(bb(inp))
%timeit out= bb(inp)
del bb

bb = up.nn.backbones.timm.TimmBackbone(name="resnet50", use_graph=True).cuda()
print_shapes(bb(inp))
%timeit out = bb(inp)
del bb

bb = up.nn.backbones.timm.TimmBackbone(name="resnet50", jit_script=True, use_graph=True).cuda()
print_shapes(bb(inp))
%timeit out = bb(inp)
del bb

bb = torch.compile(
    up.nn.backbones.timm.TimmBackbone(name="resnet50", jit_script=True, use_graph=True).cuda(), mode="max-autotune"
)
print_shapes(bb(inp))
%timeit out = bb(inp)
del bb

bb = up.nn.backbones.torchvision.TorchvisionBackbone(name="resnet50").cuda()
print_shapes(bb(inp))
%timeit out = bb(inp)
del bb

ext.1 torch.Size([16, 64, 256, 128])
ext.2 torch.Size([16, 256, 128, 64])
ext.3 torch.Size([16, 512, 64, 32])
ext.4 torch.Size([16, 1024, 32, 16])
ext.5 torch.Size([16, 2048, 16, 8])
59 ms ± 107 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
ext.1 torch.Size([16, 64, 256, 128])
ext.2 torch.Size([16, 256, 128, 64])
ext.3 torch.Size([16, 512, 64, 32])
ext.4 torch.Size([16, 1024, 32, 16])
ext.5 torch.Size([16, 2048, 16, 8])
63.3 ms ± 86 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
ext.1 torch.Size([16, 64, 256, 128])
ext.2 torch.Size([16, 256, 128, 64])
ext.3 torch.Size([16, 512, 64, 32])
ext.4 torch.Size([16, 1024, 32, 16])
ext.5 torch.Size([16, 2048, 16, 8])
59.2 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
ext.1 torch.Size([16, 64, 256, 128])
ext.2 torch.Size([16, 256, 128, 64])
ext.3 torch.Size([16, 512, 64, 32])
ext.4 torch.Size([16, 1024, 32, 16])
ext.5 torch.Size([16, 2048, 16, 8])
59.3 ms ± 122 µs per loop (mean ± std. dev. of 7 runs, 10 loo

## Feature Pyramid Networks (FPN)


In [ ]:
from unipercept.nn import backbones
from pprint import pprint

for builder in (backbones.fpn.build_default_routing, backbones.fpn.build_pan_routing, backbones.fpn.build_quad_routing):
    print(builder.__name__)
    pprint(builder(5, "sum"))

build_default_routing
{'nodes': [{'in_offsets': [3, 4],
            'level': 3,
            'weight_method': <WeightMethod.SUM: 'sum'>},
           {'in_offsets': [2, 5],
            'level': 2,
            'weight_method': <WeightMethod.SUM: 'sum'>},
           {'in_offsets': [1, 6],
            'level': 1,
            'weight_method': <WeightMethod.SUM: 'sum'>},
           {'in_offsets': [0, 7],
            'level': 0,
            'weight_method': <WeightMethod.SUM: 'sum'>},
           {'in_offsets': [1, 7, 8],
            'level': 1,
            'weight_method': <WeightMethod.SUM: 'sum'>},
           {'in_offsets': [2, 6, 9],
            'level': 2,
            'weight_method': <WeightMethod.SUM: 'sum'>},
           {'in_offsets': [3, 5, 10],
            'level': 3,
            'weight_method': <WeightMethod.SUM: 'sum'>},
           {'in_offsets': [4, 11],
            'level': 4,
            'weight_method': <WeightMethod.SUM: 'sum'>}],
 'num_levels': 5}
build_pan_routing
{'nodes': 

In [9]:
import unipercept.nn.backbones.fpn_legacy
import unipercept.nn.backbones
from pprint import pprint

bb = unipercept.nn.backbones.timm.TimmBackbone("resnet50d")

print("Feature info:")
pprint(bb.feature_info)

routing = unipercept.nn.backbones.fpn_legacy.build_quad_routing(6, "fastattn")

print("FPN routing:")
pprint(routing)

bb_fpn = unipercept.nn.backbones.fpn_legacy.FeaturePyramidBackbone(
    bb, in_features=["ext_2", "ext_3", "ext_4", "ext_5"], out_channels=64, routing=routing
)

print("FPN backbone feature info:")
pprint(bb_fpn.feature_info)

Feature info:
{'ext.1': BackboneFeatureInfo(channels=64, stride=2),
 'ext.2': BackboneFeatureInfo(channels=256, stride=4),
 'ext.3': BackboneFeatureInfo(channels=512, stride=8),
 'ext.4': BackboneFeatureInfo(channels=1024, stride=16),
 'ext.5': BackboneFeatureInfo(channels=2048, stride=32)}
FPN routing:
{'nodes': [{'in_offsets': [4, 5],
            'level': 4,
            'weight_method': <WeightMethod.FAST_ATTENTION: 'fastattn'>},
           {'in_offsets': [3, 6],
            'level': 3,
            'weight_method': <WeightMethod.FAST_ATTENTION: 'fastattn'>},
           {'in_offsets': [2, 7],
            'level': 2,
            'weight_method': <WeightMethod.FAST_ATTENTION: 'fastattn'>},
           {'in_offsets': [1, 8],
            'level': 1,
            'weight_method': <WeightMethod.FAST_ATTENTION: 'fastattn'>},
           {'in_offsets': [0, 9],
            'level': 0,
            'weight_method': <WeightMethod.FAST_ATTENTION: 'fastattn'>},
           {'in_offsets': [1, 9, 10],
  